In [2]:
import flapjack
from flapjack import registry as reg
from flapjack import tables
from flapjack import analysis
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
from pandas import Series
import numpy as np
import scipy
from scipy.optimize import curve_fit
%matplotlib inline
import pandas.io.sql as psql

In [2]:
# Replace user/password with your own
registry = reg.Registry(database_name='registry', username='postgres', password='chk77EN2018')

In [3]:
registry.load_from_file(path='/home/timrudge/Code/flapjack/datafiles/synergy/to_upload/')

Reading files from /home/timrudge/Code/flapjack/datafiles/synergy/to_upload//
--------------------------------------------------------------------


/opt/anaconda3/lib/python3.6/site-packages/openpyxl/reader/worksheet.py:318: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


Uploading data from file /home/timrudge/Code/flapjack/datafiles/synergy/to_upload/atc-iptg rep 2 24-2-18 base de datos.xlsx
Loading metadata...
Loading measurements...
--------------------------------------------------------------------
--------------------------------------------------------------------
Uploading data from file /home/timrudge/Code/flapjack/datafiles/synergy/to_upload/atc-iptg rep 3 27-2-18 base de datos.xlsx
Loading metadata...
Loading measurements...
--------------------------------------------------------------------
--------------------------------------------------------------------
Uploading data from file /home/timrudge/Code/flapjack/datafiles/synergy/to_upload/atc-iptg rep 1 23-02-18 base de datos.xlsx


KeyError: 'Worksheet Data does not exist.'

We can now get a pandas dataframe with all the data for a particular plasmid:

In [ ]:
registry.load_from_file(path='/home/timrudge/Code/flapjack/datafiles/bmg/to_upload/', fileformat='bmg')

In [ ]:
from sqlalchemy import func
existing_expt_names = psql.read_sql_table('dna', registry.engine).name.values
print(existing_expt_names)

session = registry.session()

qsamples = session.query(tables.sample.id) \
        .join(tables.vector) \
        .join(tables.dna) \
        .filter(tables.dna.name.in_(('pTet:RFP/pLacI:YFP/std:CFP','Accessory'))) \
        .group_by(tables.sample.id) \
        .having(func.count()==2)

qsamples = session.query(qsamples.subquery()).join(tables.supplement)

#q = q.join(tables.supplement) \
#        .filter(tables.supplement.name=='IPTG') \
#        .filter(tables.supplement.concentration==0) 
    
r = session.query(tables.measurement, tables.supplement) \
                    .filter(tables.measurement.sample_id.in_(qsamples)) \
                    .filter(tables.measurement.sample_id==tables.supplement.sample_id) \
                    .filter(tables.supplement.name=='IPTG') #\
                    #.filter(tables.supplement.concentration=='0.0')

df = pd.read_sql_query(r.selectable, registry.engine)

pd.set_option('display.max_rows', 200)
g = df.groupby(df.supplement_concentration)
dI = {}
mnames = df['measurement_name'].unique()
for m in mnames:
    dI[m] = np.array(())
for conc,supp in g:    
    gg = supp.groupby(supp.measurement_sample_id)
    for id,samp in gg:
        ggg = samp.groupby(samp.measurement_name)
        for mname,meas in ggg:
            #s = meas[meas.measurement_name=='RFP-YFP:500/27,540/25']
            ts = pd.Series(meas['measurement_value'].values, index=meas['measurement_time'])
            dts = ts.diff()
            np.append(dI[mname],dts.values[1:])
            plt.plot(conc, dts.max(), 'x')
        #ggg = samp.groupby(samp.supplement_concentration)
        #yfp = samp[samp.measurement_name=='RFP-YFP:500/27,540/25']['measurement_value']
        #t = samp[samp.measurement_name=='RFP-YFP:500/27,540/25']['measurement_time']
        #plt.plot(t, diff,'x')
        #for mname,meas in ggg:
        #    plt.plot(meas['measurement_time'], meas['measurement_value'])
        
#dI = np.array(dI)
from flapjack.analysis import entropy
density = entropy.prob_density(dI['RFP-YFP:500/27,540/25'], None, None)
H = entropy.entropy(density)
print(H)

In [ ]:
session = registry.session()

q = registry.query(session)
#                    .filter(tables.plasmids.name=='std:RFP/std:YFP/std:CFP')
#                    .filter(tables.experiments.name=='080313.xlsx')
#                    .filter(tables.samples.media=='M9-glucosa')\


df = registry.get_data(session, q)
df.set_index('measurement_time')
s = pd.Series(df['measurement_value'].values, index=df['measurement_time'].values)
s.plot()
print(s)

print(df.columns)
return

mnames = ['RFP-YFP:585/10,620/15', 'RFP-YFP:500/27,540/25']
#mnames = ['YFP', 'CFP']
m1,m2 = mnames

leg = []
groups = df.groupby('supplement_concentration')
for s,df in groups:
#if p==0.0:
    pg = df.groupby('dna_name')
    for p,dfp in pg:
        if p=='pTetR:RFP/pLacI:YFP/std:CFP':
            diff1,vals1,t1 = analysis.compute_measurement_diff(dfp, m1)
            diff2,vals2,t2 = analysis.compute_measurement_diff(dfp, m2)
            plt.plot(diff1, diff2, '.')
            print(p)
    leg.append(s)
    #print(type(s))

#plt.legend(leg, loc='upper left')

#print(registry.possible_values(dfatc, 'inducers_name_y'))

plt.title('std:RFP/std:YFP/std:CFP Glucosa 0,4%',fontsize=15)
plt.ylabel('dF')
plt.xlabel('Time (hours)')
#plt.xlim([0,20000])
#plt.ylim([0,20000])

#fig.savefig(fname='diffs_example-glucosa.png',dpi=1200,facecolor='w',edgecolor='w',bbox_inches='tight',pad_inches=0.2)


In [ ]:
pd.set_option('display.max_columns', 100)
atcgrp = df[df.aTc>=0].groupby('aTc')
for id,d in atcgrp:
    print(id)
    smpgrp = d.groupby('samples_id')
    for id,s in smpgrp:
        print(id)
    #print(d.head())

#df[df.aTc>=0].head()

In [ ]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import seaborn as sbn
from scipy.signal import medfilt 
from scipy.interpolate import UnivariateSpline as spline
from scipy.optimize import curve_fit
session = registry.session()

q = registry.query(session).filter(tables.sample.media=='M9-glucosa')

def piecewise_linear(x, x0, y0, k1, k2):
    return np.piecewise(x, [x < x0], [lambda x:k1*x + y0-k1*x0, lambda x:k2*x + y0-k2*x0])

def piecewise_linear(x, x0, x1, b, k1, k2, k3):
    condlist = [x < x0, (x >= x0) & (x < x1), x >= x1]
    funclist = [lambda x: k1*x + b, lambda x: k1*x + b + k2*(x-x0), lambda x: k1*x + b + k2*(x-x0) + k3*(x - x1)]
    return np.piecewise(x, condlist, funclist)

def linear(x, b, m):
    return m*x + b

def f(q, plasmid, m1, m2, diff):
    fig,ax = plt.subplots(1,2,figsize=(15,7))
    qep = q.filter(tables.dna.name==plasmid)
    dfplot = registry.get_data(session, qep)

    samples = dfplot.groupby('sample_id')
    for id,s in samples:
        s = s.sort_values('measurement_time')
        s1 = s[s.measurement_name==m1]['measurement_value'].values
        s2 = s[s.measurement_name==m2]['measurement_value'].values
        t = s[s.measurement_name==m2]['measurement_time'].values

        
        ax[0].plot(t,s1,'r')
        ax2 = ax[0].twinx()
        ax2.plot(t,s2,'g')
        
        if m1=='Time':
            ax[0].plot(t,s2)
        else:            
            if len(s1) and len(s2):
                p = np.polyfit(medfilt(s1,5),medfilt(s2,5),6)

                #p = np.polyfit(s1,s2,8)
                #f = np.poly1d(p)
                f = scipy.interpolate.interp1d(s1,s2)
                if diff:
                    #df = np.poly1d(np.polyder(p))
                    ax[1].plot(t[1:], np.diff(s1), 'r.')
                    ax[1].plot(t[1:], np.diff(s2), 'g.')
                    #plt.ylim(0,10)
                else:
                    x = np.linspace(np.min(s1),np.max(s1),100)
                    #pp,ee = curve_fit(piecewise_linear, x, f(x))
                    #print(pp)
                    #ax[1].plot((x-np.min(x))/np.max(x), (f(x)-np.min(f(x))/np.max(f(x)), 'r.-')
                    ax[1].plot(x, f(x), 'r.-')
                    #ax[1].plot(x, piecewise_linear(x,*pp), 'g')
                    #ax[1].plot(s1, s2, 'k')
                    #ax[1].plot(pp[0],20000,'go')
                    #sbn.kdeplot(vals1,data2=vals2,shade=True) #,clip=[0,np.max(vals1)*0.5])
    plt.xlabel(m1)
    plt.ylabel(m2)
    #plt.xlim(0,2000)
    #plt.ylim(0,2000)
    


mnames = session.query(tables.measurement.name).distinct() #registry.possible_values(df,'measurements_name')
enames = session.query(tables.experiment.name).distinct() #registry.possible_values(df,'experiments_name')
pnames = session.query(tables.dna.name).distinct() #registry.possible_values(df,'plasmids_name')

plt.figure(figsize=(12,7))
interact(f, plasmid = [p[0] for p in pnames],\
            m1 = [m[0] for m in mnames]+['Time'],\
            m2 = [m[0] for m in mnames], \
            diff = [True, False], \
            q = fixed(q))

'''
select_plasmid = widgets.Select(options=pnames)
init = select_plasmid.value
m_options = registry.possible_values(df)
selesct_m1 = widgets.Select(options=)
j = widgets.interactive(print_city, city=cityW)
i = widgets.interactive(select_city, country=scW)
display(i)
display(j)
'''


In [ ]:
session = registry.session()
dfctrl = registry.get_measurements(session, registry.query(session)\
                                          .filter(tables.plasmids.name=='None')\
                                          .filter(tables.cells.strain=='None')\
                                          .filter(tables.samples.media=='M9-glucosa'))
mnames = registry.possible_values(dfctrl, 'measurements_name')
fig = plt.figure(figsize=(12,3))
idx = 1
for m in mnames:
    plt.subplot(1,4,idx)
    idx += 1
    diff,vals,t = analysis.compute_measurement_diff(dfctrl, m)
    odctrl = dfctrl[dfctrl.measurements_name==m]
    plt.plot(t, diff, colors[m]+'.')
    mx = np.max(diff)
    plt.ylim(0,mx)
    plt.title('d'+names[m],fontsize=10)
    plt.xlabel('Time (hours)')
    
fig.savefig(fname='control_diffs.png',dpi=1200,facecolor='w',edgecolor='w',bbox_inches='tight',pad_inches=0.2)


In [ ]:
session = registry.session()

q = registry.query(session)\
                    .filter(tables.samples.media=='M9-glucosa')
df = registry.get_measurements(session, q)
#df = df[(df.supplements_concentration_y==-1)&(df.supplements_concentration_x==-1)]
plasmidnames = registry.possible_values(df, 'plasmids_name')
print(plasmidnames)
fig = plt.figure(figsize=(15,15))
plt.title('Glucosa 0,4%')
idx = 1
for p in plasmidnames:
    if 'None' not in p:
    #if ('Lac' in p or 'Tet' in p) and ('Las' not in p) and ('Lux' not in p) or p=='None':
        dfp = df[df.plasmids_name==p]

        Hcond2 = analysis.channel_conditional_entropy_2d(dfp, 'OD600:600', 'RFP-YFP:500/27,540/25', 256, [[0,0.3],[0,40000]])
        Hcond3a = analysis.channel_conditional_entropy_3d(dfp, 'OD600:600', 'RFP-YFP:500/27,540/25', 'RFP-YFP:585/10,620/15', 256, [[0,0.3],[0,40000]])
        Hcond3b = analysis.channel_conditional_entropy_3d(dfp, 'OD600:600', 'RFP-YFP:500/27,540/25', 'CFP:420/50,485/20', 256, [[0,0.3],[0,40000]])

        Hcond4 = analysis.channel_conditional_entropy_4d(dfp, 'OD600:600', 'RFP-YFP:500/27,540/25', 'RFP-YFP:585/10,620/15', 'CFP:420/50,485/20', 256, [[0,0.3],[0,40000]])

        H = analysis.channel_entropy(dfp, 'RFP-YFP:500/27,540/25', 256, [0,40000])
        '''
        print(H)
        print(Hcond2)
        print(Hcond3a)
        print(Hcond3b)
        print(Hcond4)
        print(p)
        '''

        plt.subplot(4,4,idx)
        idx += 1
        plt.ylim(ymax=1.2)
        plt.bar([0,1,2,3,4],[H/H,Hcond2/H,Hcond3a/H, Hcond3b/H, Hcond4/H ],color='y')
        plt.title(p + ' -aTc/-IPTG',fontsize=10)
        plt.xticks([0,1,2,3,4], ['0','1','2','2','3'])
        for a,b in zip([0,1,2,3,4],[H/H,Hcond2/H,Hcond3a/H, Hcond3b/H, Hcond4/H ]):
            plt.text(a-0.2,b+0.05,str(np.round(b,2)),fontsize=10.0, color='k')


In [ ]:
fig.savefig(fname='hu-atc-iptg.png',dpi=1200,facecolor='w',edgecolor='w',bbox_inches='tight',pad_inches=0.2)

print(H)
print(Hcond2)
print(Hcond3a)
print(Hcond3b)
print(Hcond4)
print(p)

In [ ]:
session = registry.session()

q = registry.query(session)\
                    .filter(tables.samples.media=='M9-glicerol')
df = registry.get_measurements(session, q)
df = df[(df.supplements_concentration_y==-1)&(df.supplements_concentration_x==-1)]
plasmidnames = registry.possible_values(df, 'plasmids_name')
print(plasmidnames)
fig = plt.figure(figsize=(15,15))
plt.title('Glucosa 0,4%')
idx = 1
for p in plasmidnames:
    if ('Lac' in p or 'Tet' in p) and ('Las' not in p) and ('Lux' not in p) or p=='None':
        dfp = df[df.plasmids_name==p]

        Hcond2 = analysis.channel_conditional_entropy_2d(dfp, 'OD600:600', 'RFP-YFP:500/27,540/25', 256, [[0,0.3],[0,40000]])
        Hcond3a = analysis.channel_conditional_entropy_3d(dfp, 'OD600:600', 'RFP-YFP:500/27,540/25', 'RFP-YFP:585/10,620/15', 256, [[0,0.3],[0,40000]])
        Hcond3b = analysis.channel_conditional_entropy_3d(dfp, 'OD600:600', 'RFP-YFP:500/27,540/25', 'CFP:420/50,485/20', 256, [[0,0.3],[0,40000]])

        Hcond4 = analysis.channel_conditional_entropy_4d(dfp, 'OD600:600', 'RFP-YFP:500/27,540/25', 'RFP-YFP:585/10,620/15', 'CFP:420/50,485/20', 256, [[0,0.3],[0,40000]])

        H = analysis.channel_entropy(dfp, 'RFP-YFP:500/27,540/25', 256, [0,40000])
        '''
        print(H)
        print(Hcond2)
        print(Hcond3a)
        print(Hcond3b)
        print(Hcond4)
        print(p)
        '''

        plt.subplot(4,4,idx)
        idx += 1
        plt.bar([0,1,2,3,4],[H,Hcond2,Hcond3a, Hcond3b, Hcond4 ],color='y')
        plt.title(p)
